<a href="https://colab.research.google.com/github/JorgeAccardi/auscultacion-presa/blob/main/Estadistica_Regresi%C3%B3n_Lineal_De_Instrumentos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import io
from IPython.display import display, clear_output
import ipywidgets as widgets

# Diccionarios para almacenar por tipo de archivo
instrumentos = [
    "puntos_fijos_mi",
    "puntos_fijos_md",
    "inclinometros",
    "asentamiento",
    "piezometros_electricos",
    "piezometros_casagrande",
    "freatimetros",
    "extensometro"
]

datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Función para detectar tipo de instrumento por nombre
def detectar_instrumento(nombre):
    nombre = nombre.lower()
    if "puntosfijos" in nombre or "pf" in nombre:
        if "mi" in nombre:
            return "puntos_fijos_mi"
        elif "md" in nombre:
            return "puntos_fijos_md"
        else:
            return None  # Puntos fijos sin margen, no válido
    elif "incli" in nombre:
        return "inclinometros"
    elif "as" in nombre:
        return "asentamiento"
    elif "pe" in nombre:
        return "piezometros_electricos"
    elif "pcg" in nombre:
        return "piezometros_casagrande"
    elif "frea" in nombre:
        return "freatimetros"
    elif "ext" in nombre:
        return "extensometro"
    return None

# --- Widget de carga de archivos ---
upload_widget = widgets.FileUpload(
    accept='.csv,.xlsx',
    multiple=True,
    description='Subir archivos',
    style={'button_color': 'lightblue'}
)

output = widgets.Output()

# Función principal de carga
def cargar_archivos(change):
    with output:
        clear_output(wait=True)
        archivos = upload_widget.value

        if not archivos:
            print("⚠️ No se subió ningún archivo.")
            return

        for nombre_archivo, archivo_info in archivos.items():
            try:
                contenido = archivo_info['content']
                extension = nombre_archivo.split('.')[-1].lower()
                instrumento = detectar_instrumento(nombre_archivo)

                if not instrumento:
                    print(f"❌ Instrumento no reconocido o mal nombrado: {nombre_archivo}")
                    continue

                # Cargar el archivo
                if extension == 'csv':
                    df = pd.read_csv(io.BytesIO(contenido), encoding='utf-8')
                    datos_csv[instrumento] = pd.concat([datos_csv[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (CSV)")
                elif extension == 'xlsx':
                    df = pd.read_excel(io.BytesIO(contenido))
                    datos_xlsx[instrumento] = pd.concat([datos_xlsx[instrumento], df], ignore_index=True)
                    print(f"✅ {nombre_archivo} → {instrumento} (XLSX)")
                else:
                    print(f"❌ Formato no compatible: {nombre_archivo}")
            except Exception as e:
                print(f"❌ Error al procesar {nombre_archivo}: {e}")

        mostrar_menu()

# Función de visualización dinámica
def mostrar_menu():
    opciones = []
    for origen in ['csv', 'xlsx']:
        for instrumento in instrumentos:
            opciones.append(f"{instrumento} ({origen})")

    selector = widgets.Dropdown(
        options=opciones,
        description='Seleccionar DataFrame:',
        style={'description_width': 'initial'},
        layout=widgets.Layout(width='60%')
    )

    def mostrar_datos(change):
        clear_output(wait=True)
        display(upload_widget, output)
        seleccion = selector.value
        instrumento, origen = seleccion.split(" ")
        instrumento = instrumento.strip()
        origen = origen.strip("()")

        print(f"📊 Mostrando: {instrumento.upper()} ({origen.upper()})")
        if origen == "csv":
            display(datos_csv[instrumento].head())
        else:
            display(datos_xlsx[instrumento].head())

    selector.observe(mostrar_datos, names='value')
    display(selector)

# Conectar evento
upload_widget.observe(cargar_archivos, names='value')

# Mostrar interfaz
display(upload_widget)
display(output)

FileUpload(value={'AS175_20250608.csv': {'metadata': {'name': 'AS175_20250608.csv', 'type': 'text/csv', 'size'…

Output()

📊 Mostrando: PUNTOS_FIJOS_MD (XLSX)


,FECHA,MARGEN,INSTRUMENTO,DELTA_NORTE_[M],DELTA_ESTE_[M],DELTA_COTA_[M],DISTANCIA_[M],DISTANCIA_(MM),AZIMUT_REF._AL_NORTE,TASA_NORTE_(MM/DIA),TASA_ESTE_(MM/DIA),TASA_COTA_(MM/DIA),TASA_DISTANCIA_(MM/DIA)
0,04/10/2021,Derecha,PF01 PRIMA,0.000333,-0.000667,0.003333,0.000745,0.745356,296.565051,0.041667,-0.083333,0.416667,0.093169
1,10/10/2021,Derecha,PF01 PRIMA,-0.000667,-0.000667,0.004333,0.000943,0.942809,225.000000,-0.047619,-0.047619,0.309524,0.067344
2,14/10/2021,Derecha,PF01 PRIMA,0.003333,0.003333,0.003333,0.004714,4.714045,45.000000,0.185185,0.185185,0.185185,0.261891
3,18/10/2021,Derecha,PF01 PRIMA,0.002333,0.002333,0.000333,0.003300,3.299832,45.000000,0.106061,0.106061,0.015152,0.149992
4,27/10/2021,Derecha,PF01 PRIMA,-0.000667,-0.000667,0.005333,0.000943,0.942809,225.000000,-0.021505,-0.021505,0.172043,0.030413


In [4]:
import pandas as pd
import numpy as np
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import statsmodels.api as sm
import matplotlib.pyplot as plt

# Asumimos que datos_csv y datos_xlsx están poblados por el script de carga
instrumentos = [
    "puntos_fijos_mi", "puntos_fijos_md", "inclinometros", "asentamiento",
    "piezometros_electricos", "piezometros_casagrande", "freatimetros", "extensometro"
]

try:
    datos_csv
except NameError:
    datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
try:
    datos_xlsx
except NameError:
    datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Definir widgets
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande", "Inclinómetros",
             "Celdas de Asentamiento", "Freatímetros", "Extensómetros"],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
margen_dropdown = widgets.Dropdown(description="Margen:", options=[])
punto_dropdown = widgets.Dropdown(description="Punto Fijo:", options=[])
variable_pf_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pf_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
progresiva_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_pe_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pe_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
margen_cg_dropdown = widgets.Dropdown(description="Margen:", options=[])
pz_cg_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_cg_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_cg_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:", options=[])
anio_inc_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
eje_dropdown = widgets.Dropdown(options=["A+", "A-", "B+", "B-"], value="A+", description="Eje:")
progresiva_ca_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
celda_dropdown = widgets.Dropdown(description="Celda:", options=[])
variable_ca_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ca_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
freatimetro_dropdown = widgets.Dropdown(description="Freatímetro:", options=[])
variable_fr_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_fr_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
extensometro_dropdown = widgets.Dropdown(description="Extensómetro:", options=[])
variable_ex_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ex_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

# Botones y salida
boton_estadisticas = widgets.Button(description="Calcular Estadísticas", button_style="primary")
boton_regresion = widgets.Button(description="Realizar Regresión", button_style="success")
output = widgets.Output()

# Funciones para actualizar opciones de dropdowns
def actualizar_opciones_pf(change=None):
    origen = origen_dropdown.value
    df_mi = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
    df_md = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
    datasets = {"Margen Izquierda (MI)": df_mi, "Margen Derecha (MD)": df_md}
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets:
        margen_dropdown.options = []
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = ["Todos"]
        return
    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    if 'FECHA' not in df.columns:
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = ["Todos"]
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variable_pf_dropdown.options = [col for col in df.select_dtypes(include='number').columns if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique().tolist())
    anio_pf_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_opciones_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'FECHA' not in df.columns:
        progresiva_dropdown.options = []
        piezometro_dropdown.options = []
        variable_pe_dropdown.options = []
        anio_pe_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique().tolist())
    if progresiva_dropdown.options:
        progresiva_dropdown.value = progresiva_dropdown.options[0]
    actualizar_piezometros_pe()
    variable_pe_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']]
    anio_pe_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'PIEZOMETRO' not in df.columns:
        piezometro_dropdown.options = []
        return
    piezos = sorted(df[df['PROGRESIVA'] == progresiva_dropdown.value]['PIEZOMETRO'].dropna().unique().tolist())
    piezometro_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    if df.empty or 'MARGEN' not in df.columns or 'FECHA' not in df.columns:
        margen_cg_dropdown.options = []
        pz_cg_dropdown.options = []
        variable_cg_dropdown.options = []
        anio_cg_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margen_cg_dropdown.options = sorted(df['MARGEN'].dropna().unique().tolist())
    if margen_cg_dropdown.options:
        margen_cg_dropdown.value = margen_cg_dropdown.options[0]
    actualizar_piezometros_cg()
    variable_cg_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'MARGEN', 'PIEZOMETRO']]
    anio_cg_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    if df.empty or 'MARGEN' not in df.columns or 'PIEZOMETRO' not in df.columns:
        pz_cg_dropdown.options = []
        return
    piezos = sorted(df[df['MARGEN'] == margen_cg_dropdown.value]['PIEZOMETRO'].dropna().unique().tolist())
    pz_cg_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_inc(change=None):
    df = datos_csv["inclinometros"] if origen_dropdown.value == "CSV" else datos_xlsx["inclinometros"]
    if df.empty or 'Inclinometro' not in df.columns or 'Fecha' not in df.columns:
        inclinometro_dropdown.options = []
        anio_inc_dropdown.options = ["Todos"]
        eje_dropdown.options = ["A+", "A-", "B+", "B-"]
        return
    df = df.copy()
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    inclinometro_dropdown.options = ["Todos"] + sorted(df['Inclinometro'].dropna().unique().tolist())
    anio_inc_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['Fecha'].dt.year.dropna().unique())]
    eje_dropdown.options = ["A+", "A-", "B+", "B-"]

def actualizar_opciones_ca(change=None):
    df = datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'FECHA' not in df.columns:
        progresiva_ca_dropdown.options = []
        celda_dropdown.options = []
        variable_ca_dropdown.options = []
        anio_ca_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_ca_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique().tolist())
    if progresiva_ca_dropdown.options:
        progresiva_ca_dropdown.value = progresiva_ca_dropdown.options[0]
    actualizar_celdas_ca()
    variable_ca_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'CELDA_DE_ASENTAMIENTO']]
    anio_ca_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_celdas_ca(change=None):
    df = datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'CELDA_DE_ASENTAMIENTO' not in df.columns:
        celda_dropdown.options = []
        return
    celdas = sorted(df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]['CELDA_DE_ASENTAMIENTO'].dropna().unique().tolist())
    celda_dropdown.options = ["Todas"] + list(celdas)

def actualizar_opciones_fr(change=None):
    df = datos_csv["freatimetros"] if origen_dropdown.value == "CSV" else datos_xlsx["freatimetros"]
    if df.empty or 'FREATIMETRO' not in df.columns or 'FECHA' not in df.columns:
        freatimetro_dropdown.options = []
        variable_fr_dropdown.options = []
        anio_fr_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    freatimetro_dropdown.options = ["Todos"] + sorted(df['FREATIMETRO'].dropna().unique().tolist())
    variable_fr_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'FREATIMETRO']]
    anio_fr_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_opciones_ex(change=None):
    df = datos_csv["extensometro"] if origen_dropdown.value == "CSV" else datos_xlsx["extensometro"]
    if df.empty or 'EXTENSOMETRO' not in df.columns or 'FECHA' not in df.columns:
        extensometro_dropdown.options = []
        variable_ex_dropdown.options = []
        anio_ex_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    extensometro_dropdown.options = ["Todos"] + sorted(df['EXTENSOMETRO'].dropna().unique().tolist())
    variable_ex_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'EXTENSOMETRO']]
    anio_ex_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# Función para mostrar/ocultar controles
def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value
    for w in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
              progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown,
              margen_cg_dropdown, pz_cg_dropdown, variable_cg_dropdown, anio_cg_dropdown,
              inclinometro_dropdown, anio_inc_dropdown, eje_dropdown,
              progresiva_ca_dropdown, celda_dropdown, variable_ca_dropdown, anio_ca_dropdown,
              freatimetro_dropdown, variable_fr_dropdown, anio_fr_dropdown,
              extensometro_dropdown, variable_ex_dropdown, anio_ex_dropdown]:
        w.layout.display = 'none'
    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    elif tipo == "Piezómetros Eléctricos":
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()
    elif tipo == "Piezómetros Casagrande":
        margen_cg_dropdown.layout.display = 'flex'
        pz_cg_dropdown.layout.display = 'flex'
        variable_cg_dropdown.layout.display = 'flex'
        anio_cg_dropdown.layout.display = 'flex'
        actualizar_opciones_cg()
    elif tipo == "Inclinómetros":
        inclinometro_dropdown.layout.display = 'flex'
        anio_inc_dropdown.layout.display = 'flex'
        eje_dropdown.layout.display = 'flex'
        actualizar_opciones_inc()
    elif tipo == "Celdas de Asentamiento":
        progresiva_ca_dropdown.layout.display = 'flex'
        celda_dropdown.layout.display = 'flex'
        variable_ca_dropdown.layout.display = 'flex'
        anio_ca_dropdown.layout.display = 'flex'
        actualizar_opciones_ca()
    elif tipo == "Freatímetros":
        freatimetro_dropdown.layout.display = 'flex'
        variable_fr_dropdown.layout.display = 'flex'
        anio_fr_dropdown.layout.display = 'flex'
        actualizar_opciones_fr()
    elif tipo == "Extensómetros":
        extensometro_dropdown.layout.display = 'flex'
        variable_ex_dropdown.layout.display = 'flex'
        anio_ex_dropdown.layout.display = 'flex'
        actualizar_opciones_ex()

# Función para calcular estadísticas
def calcular_estadisticas(b=None):
    with output:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            col = 'INSTRUMENTO'
            df = df.copy()
            if anio_pf_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value]
            titulo = f"{margen_dropdown.value}: {variable}"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Eléctricos.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_pe_dropdown.value
            col = 'PIEZOMETRO'
            if progresiva_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
            if anio_pe_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
            titulo = f"{progresiva_dropdown.value} – {variable}"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Casagrande.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_cg_dropdown.value
            col = 'PIEZOMETRO'
            if margen_cg_dropdown.value != "Todos":
                df = df[df['MARGEN'] == margen_cg_dropdown.value]
            if pz_cg_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
            if anio_cg_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
            titulo = f"{margen_cg_dropdown.value} – {variable}"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Inclinómetros.")
                return
            df = df.copy()
            df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
            variable = eje_dropdown.value
            col = 'Inclinometro'
            if inclinometro_dropdown.value != "Todos":
                df = df[df['Inclinometro'] == inclinometro_dropdown.value]
            if anio_inc_dropdown.value != "Todos":
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)]
            titulo = f"{inclinometro_dropdown.value} – {variable}"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Celdas de Asentamiento.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_ca_dropdown.value
            col = 'CELDA_DE_ASENTAMIENTO'
            if progresiva_ca_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]
            if celda_dropdown.value != "Todas":
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value]
            if anio_ca_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)]
            titulo = f"{progresiva_ca_dropdown.value} – {variable}"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Freatímetros.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_fr_dropdown.value
            col = 'FREATIMETRO'
            if freatimetro_dropdown.value != "Todos":
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value]
            if anio_fr_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)]
            titulo = f"{freatimetro_dropdown.value} – {variable}"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Extensómetros.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_ex_dropdown.value
            col = 'EXTENSOMETRO'
            if extensometro_dropdown.value != "Todos":
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value]
            if anio_ex_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)]
            titulo = f"{extensometro_dropdown.value} – {variable}"
        else:
            print("⚠️ Instrumento no reconocido.")
            return

        if variable not in df.columns:
            print(f"⚠️ La variable {variable} no está disponible en los datos.")
            return

        df[variable] = pd.to_numeric(df[variable], errors='coerce')
        df = df.dropna(subset=[variable])
        if df.empty:
            print(f"⚠️ No hay datos válidos para la variable {variable} después de filtrar.")
            return

        datos = df[variable]
        media = datos.mean()
        mediana = datos.median()
        moda = stats.mode(datos, keepdims=True)[0][0] if len(datos) > 0 else np.nan
        varianza = datos.var()
        desv_est = datos.std()
        minimo = datos.min()
        maximo = datos.max()
        conteo = datos.count()

        print(f"📊 Estadísticas Descriptivas para {titulo}")
        print(f"Conteo: {conteo}")
        print(f"Media: {media:.2f}")
        print(f"Mediana: {mediana:.2f}")
        print(f"Moda: {moda:.2f}")
        print(f"Varianza: {varianza:.2f}")
        print(f"Desviación Estándar: {desv_est:.2f}")
        print(f"Mínimo: {minimo:.2f}")
        print(f"Máximo: {maximo:.2f}")

# Función para regresión lineal
def realizar_regresion(b=None):
    with output:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        fecha_col = 'FECHA'  # Default para la mayoría
        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            col = 'INSTRUMENTO'
            df = df.copy()
            if anio_pf_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value]
            titulo = f"{margen_dropdown.value}: {variable} vs. Tiempo"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Eléctricos.")
                return
            df = df.copy()
            variable = variable_pe_dropdown.value
            col = 'PIEZOMETRO'
            if progresiva_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
            if anio_pe_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
            titulo = f"{progresiva_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Casagrande.")
                return
            df = df.copy()
            variable = variable_cg_dropdown.value
            col = 'PIEZOMETRO'
            if margen_cg_dropdown.value != "Todos":
                df = df[df['MARGEN'] == margen_cg_dropdown.value]
            if pz_cg_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
            if anio_cg_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
            titulo = f"{margen_cg_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Inclinómetros.")
                return
            df = df.copy()
            fecha_col = 'Fecha'  # Inclinómetros usa 'Fecha'
            variable = eje_dropdown.value
            col = 'Inclinometro'
            if inclinometro_dropdown.value != "Todos":
                df = df[df['Inclinometro'] == inclinometro_dropdown.value]
            if anio_inc_dropdown.value != "Todos":
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)]
            titulo = f"{inclinometro_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Celdas de Asentamiento.")
                return
            df = df.copy()
            variable = variable_ca_dropdown.value
            col = 'CELDA_DE_ASENTAMIENTO'
            if progresiva_ca_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]
            if celda_dropdown.value != "Todas":
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value]
            if anio_ca_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)]
            titulo = f"{progresiva_ca_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Freatímetros.")
                return
            df = df.copy()
            variable = variable_fr_dropdown.value
            col = 'FREATIMETRO'
            if freatimetro_dropdown.value != "Todos":
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value]
            if anio_fr_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)]
            titulo = f"{freatimetro_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Extensómetros.")
                return
            df = df.copy()
            variable = variable_ex_dropdown.value
            col = 'EXTENSOMETRO'
            if extensometro_dropdown.value != "Todos":
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value]
            if anio_ex_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)]
            titulo = f"{extensometro_dropdown.value} – {variable} vs. Tiempo"
        else:
            print("⚠️ Instrumento no reconocido.")
            return

        if variable not in df.columns or fecha_col not in df.columns:
            print(f"⚠️ La variable {variable} o la columna de fecha no están disponibles en los datos.")
            return

        df[variable] = pd.to_numeric(df[variable], errors='coerce')
        df[fecha_col] = pd.to_datetime(df[fecha_col], dayfirst=True, errors='coerce')
        df = df.dropna(subset=[variable, fecha_col])
        if df.empty or len(df) < 2:
            print(f"⚠️ No hay suficientes datos válidos para la regresión de {variable}.")
            return

        # Normalizar Dias para mejorar estabilidad numérica
        df['Dias'] = (df[fecha_col] - df[fecha_col].min()).dt.total_seconds() / (24 * 3600)
        if df['Dias'].std() > 0:
            df['Dias'] = (df['Dias'] - df['Dias'].mean()) / df['Dias'].std()
        else:
            df['Dias'] = df['Dias'] - df['Dias'].mean()  # Centrar si std es 0
        X = sm.add_constant(df['Dias'])
        y = df[variable]

        # Ajustar modelo
        try:
            model = sm.OLS(y, X).fit()
        except Exception as e:
            print(f"❌ Error al ajustar el modelo de regresión: {e}")
            return

        # Predicciones
        y_pred = model.predict(X)

        # Mostrar resultados
        print(f"📈 Regresión Lineal para {titulo}")
        print(model.summary())

        # Visualización
        plt.figure(figsize=(10, 6))
        plt.scatter(df[fecha_col], y, label='Datos', color='blue')
        plt.plot(df[fecha_col], y_pred, label='Regresión', color='red')
        plt.xlabel('Fecha')
        plt.ylabel(variable)
        plt.title(titulo)
        plt.legend()
        plt.grid(True)
        plt.show()

# Conectar eventos
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
margen_dropdown.observe(actualizar_opciones_pf, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
margen_cg_dropdown.observe(actualizar_piezometros_cg, names='value')
progresiva_ca_dropdown.observe(actualizar_celdas_ca, names='value')
boton_estadisticas.on_click(calcular_estadisticas)
boton_regresion.on_click(realizar_regresion)

# Mostrar interfaz
display(HTML("<h3 style='color:#1866a3'>Análisis Estadístico y Regresión</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))
display(widgets.HBox([
    margen_dropdown, punto_dropdown,
    progresiva_dropdown, piezometro_dropdown,
    margen_cg_dropdown, pz_cg_dropdown,
    inclinometro_dropdown,
    progresiva_ca_dropdown, celda_dropdown,
    freatimetro_dropdown,
    extensometro_dropdown
]))
display(widgets.HBox([
    variable_pf_dropdown, variable_pe_dropdown, variable_cg_dropdown,
    anio_pf_dropdown, anio_pe_dropdown, anio_cg_dropdown,
    anio_inc_dropdown, eje_dropdown,
    variable_ca_dropdown, anio_ca_dropdown,
    variable_fr_dropdown, anio_fr_dropdown,
    variable_ex_dropdown, anio_ex_dropdown
]))
display(widgets.HBox([boton_estadisticas, boton_regresion]))
display(output)

# Inicializar controles
actualizar_controles_visibles()

Output()

In [10]:
import pandas as pd
import numpy as np
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns  # Nueva importación para visualizaciones

# Asumimos que datos_csv y datos_xlsx están poblados por el script de carga
instrumentos = [
    "puntos_fijos_mi", "puntos_fijos_md", "inclinometros", "asentamiento",
    "piezometros_electricos", "piezometros_casagrande", "freatimetros", "extensometro"
]

try:
    datos_csv
except NameError:
    datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
try:
    datos_xlsx
except NameError:
    datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Definir widgets
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande", "Inclinómetros",
             "Celdas de Asentamiento", "Freatímetros", "Extensómetros"],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
margen_dropdown = widgets.Dropdown(description="Margen:", options=[])
punto_dropdown = widgets.Dropdown(description="Punto Fijo:", options=[])
variable_pf_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pf_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
progresiva_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_pe_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pe_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
margen_cg_dropdown = widgets.Dropdown(description="Margen:", options=[])
pz_cg_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_cg_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_cg_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:", options=[])
anio_inc_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
eje_dropdown = widgets.Dropdown(options=["A+", "A-", "B+", "B-"], value="A+", description="Eje:")
progresiva_ca_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
celda_dropdown = widgets.Dropdown(description="Celda:", options=[])
variable_ca_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ca_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
freatimetro_dropdown = widgets.Dropdown(description="Freatímetro:", options=[])
variable_fr_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_fr_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
extensometro_dropdown = widgets.Dropdown(description="Extensómetro:", options=[])
variable_ex_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ex_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

# Botones y salida
boton_estadisticas = widgets.Button(description="Calcular Estadísticas", button_style="primary")
boton_regresion = widgets.Button(description="Realizar Regresión", button_style="success")
boton_visualizar = widgets.Button(description="Visualizar Distribución", button_style="info")  # Nuevo botón
output = widgets.Output()

# Funciones para actualizar opciones de dropdowns
def actualizar_opciones_pf(change=None):
    origen = origen_dropdown.value
    df_mi = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
    df_md = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
    datasets = {"Margen Izquierda (MI)": df_mi, "Margen Derecha (MD)": df_md}
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets:
        margen_dropdown.options = []
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = ["Todos"]
        return
    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    if 'FECHA' not in df.columns:
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = ["Todos"]
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variable_pf_dropdown.options = [col for col in df.select_dtypes(include='number').columns if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique().tolist())
    anio_pf_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_opciones_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'FECHA' not in df.columns:
        progresiva_dropdown.options = []
        piezometro_dropdown.options = []
        variable_pe_dropdown.options = []
        anio_pe_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique().tolist())
    if progresiva_dropdown.options:
        progresiva_dropdown.value = progresiva_dropdown.options[0]
    actualizar_piezometros_pe()
    variable_pe_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']]
    anio_pe_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'PIEZOMETRO' not in df.columns:
        piezometro_dropdown.options = []
        return
    piezos = sorted(df[df['PROGRESIVA'] == progresiva_dropdown.value]['PIEZOMETRO'].dropna().unique().tolist())
    piezometro_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    if df.empty or 'MARGEN' not in df.columns or 'FECHA' not in df.columns:
        margen_cg_dropdown.options = []
        pz_cg_dropdown.options = []
        variable_cg_dropdown.options = []
        anio_cg_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margen_cg_dropdown.options = sorted(df['MARGEN'].dropna().unique().tolist())
    if margen_cg_dropdown.options:
        margen_cg_dropdown.value = margen_cg_dropdown.options[0]
    actualizar_piezometros_cg()
    variable_cg_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'MARGEN', 'PIEZOMETRO']]
    anio_cg_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    if df.empty or 'MARGEN' not in df.columns or 'PIEZOMETRO' not in df.columns:
        pz_cg_dropdown.options = []
        return
    piezos = sorted(df[df['MARGEN'] == margen_cg_dropdown.value]['PIEZOMETRO'].dropna().unique().tolist())
    pz_cg_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_inc(change=None):
    df = datos_csv["inclinometros"] if origen_dropdown.value == "CSV" else datos_xlsx["inclinometros"]
    if df.empty or 'Inclinometro' not in df.columns or 'Fecha' not in df.columns:
        inclinometro_dropdown.options = []
        anio_inc_dropdown.options = ["Todos"]
        eje_dropdown.options = ["A+", "A-", "B+", "B-"]
        return
    df = df.copy()
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    inclinometro_dropdown.options = ["Todos"] + sorted(df['Inclinometro'].dropna().unique().tolist())
    anio_inc_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['Fecha'].dt.year.dropna().unique())]
    eje_dropdown.options = ["A+", "A-", "B+", "B-"]

def actualizar_opciones_ca(change=None):
    df = datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'FECHA' not in df.columns:
        progresiva_ca_dropdown.options = []
        celda_dropdown.options = []
        variable_ca_dropdown.options = []
        anio_ca_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_ca_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique().tolist())
    if progresiva_ca_dropdown.options:
        progresiva_ca_dropdown.value = progresiva_ca_dropdown.options[0]
    actualizar_celdas_ca()
    variable_ca_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'CELDA_DE_ASENTAMIENTO']]
    anio_ca_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_celdas_ca(change=None):
    df = datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'CELDA_DE_ASENTAMIENTO' not in df.columns:
        celda_dropdown.options = []
        return
    celdas = sorted(df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]['CELDA_DE_ASENTAMIENTO'].dropna().unique().tolist())
    celda_dropdown.options = ["Todas"] + list(celdas)

def actualizar_opciones_fr(change=None):
    df = datos_csv["freatimetros"] if origen_dropdown.value == "CSV" else datos_xlsx["freatimetros"]
    if df.empty or 'FREATIMETRO' not in df.columns or 'FECHA' not in df.columns:
        freatimetro_dropdown.options = []
        variable_fr_dropdown.options = []
        anio_fr_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    freatimetro_dropdown.options = ["Todos"] + sorted(df['FREATIMETRO'].dropna().unique().tolist())
    variable_fr_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'FREATIMETRO']]
    anio_fr_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_opciones_ex(change=None):
    df = datos_csv["extensometro"] if origen_dropdown.value == "CSV" else datos_xlsx["extensometro"]
    if df.empty or 'EXTENSOMETRO' not in df.columns or 'FECHA' not in df.columns:
        extensometro_dropdown.options = []
        variable_ex_dropdown.options = []
        anio_ex_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    extensometro_dropdown.options = ["Todos"] + sorted(df['EXTENSOMETRO'].dropna().unique().tolist())
    variable_ex_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'EXTENSOMETRO']]
    anio_ex_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# Función para mostrar/ocultar controles
def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value
    for w in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
              progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, anio_pe_dropdown,
              margen_cg_dropdown, pz_cg_dropdown, variable_cg_dropdown, anio_cg_dropdown,
              inclinometro_dropdown, anio_inc_dropdown, eje_dropdown,
              progresiva_ca_dropdown, celda_dropdown, variable_ca_dropdown, anio_ca_dropdown,
              freatimetro_dropdown, variable_fr_dropdown, anio_fr_dropdown,
              extensometro_dropdown, variable_ex_dropdown, anio_ex_dropdown]:
        w.layout.display = 'none'
    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    elif tipo == "Piezómetros Eléctricos":
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()
    elif tipo == "Piezómetros Casagrande":
        margen_cg_dropdown.layout.display = 'flex'
        pz_cg_dropdown.layout.display = 'flex'
        variable_cg_dropdown.layout.display = 'flex'
        anio_cg_dropdown.layout.display = 'flex'
        actualizar_opciones_cg()
    elif tipo == "Inclinómetros":
        inclinometro_dropdown.layout.display = 'flex'
        anio_inc_dropdown.layout.display = 'flex'
        eje_dropdown.layout.display = 'flex'
        actualizar_opciones_inc()
    elif tipo == "Celdas de Asentamiento":
        progresiva_ca_dropdown.layout.display = 'flex'
        celda_dropdown.layout.display = 'flex'
        variable_ca_dropdown.layout.display = 'flex'
        anio_ca_dropdown.layout.display = 'flex'
        actualizar_opciones_ca()
    elif tipo == "Freatímetros":
        freatimetro_dropdown.layout.display = 'flex'
        variable_fr_dropdown.layout.display = 'flex'
        anio_fr_dropdown.layout.display = 'flex'
        actualizar_opciones_fr()
    elif tipo == "Extensómetros":
        extensometro_dropdown.layout.display = 'flex'
        variable_ex_dropdown.layout.display = 'flex'
        anio_ex_dropdown.layout.display = 'flex'
        actualizar_opciones_ex()

# Función para calcular estadísticas
def calcular_estadisticas(b=None):
    with output:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            col = 'INSTRUMENTO'
            df = df.copy()
            if anio_pf_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value]
            titulo = f"{margen_dropdown.value}: {variable}"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Eléctricos.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_pe_dropdown.value
            col = 'PIEZOMETRO'
            if progresiva_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
            if anio_pe_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
            titulo = f"{progresiva_dropdown.value} – {variable}"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Casagrande.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_cg_dropdown.value
            col = 'PIEZOMETRO'
            if margen_cg_dropdown.value != "Todos":
                df = df[df['MARGEN'] == margen_cg_dropdown.value]
            if pz_cg_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
            if anio_cg_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
            titulo = f"{margen_cg_dropdown.value} – {variable}"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Inclinómetros.")
                return
            df = df.copy()
            df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
            variable = eje_dropdown.value
            col = 'Inclinometro'
            if inclinometro_dropdown.value != "Todos":
                df = df[df['Inclinometro'] == inclinometro_dropdown.value]
            if anio_inc_dropdown.value != "Todos":
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)]
            titulo = f"{inclinometro_dropdown.value} – {variable}"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Celdas de Asentamiento.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_ca_dropdown.value
            col = 'CELDA_DE_ASENTAMIENTO'
            if progresiva_ca_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]
            if celda_dropdown.value != "Todas":
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value]
            if anio_ca_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)]
            titulo = f"{progresiva_ca_dropdown.value} – {variable}"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Freatímetros.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_fr_dropdown.value
            col = 'FREATIMETRO'
            if freatimetro_dropdown.value != "Todos":
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value]
            if anio_fr_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)]
            titulo = f"{freatimetro_dropdown.value} – {variable}"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Extensómetros.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_ex_dropdown.value
            col = 'EXTENSOMETRO'
            if extensometro_dropdown.value != "Todos":
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value]
            if anio_ex_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)]
            titulo = f"{extensometro_dropdown.value} – {variable}"
        else:
            print("⚠️ Instrumento no reconocido.")
            return

        if variable not in df.columns:
            print(f"⚠️ La variable {variable} no está disponible en los datos.")
            return

        df[variable] = pd.to_numeric(df[variable], errors='coerce')
        df = df.dropna(subset=[variable])
        if df.empty:
            print(f"⚠️ No hay datos válidos para la variable {variable} después de filtrar.")
            return

        datos = df[variable]
        media = datos.mean()
        mediana = datos.median()
        moda = stats.mode(datos, keepdims=True)[0][0] if len(datos) > 0 else np.nan
        varianza = datos.var()
        desv_est = datos.std()
        minimo = datos.min()
        maximo = datos.max()
        conteo = datos.count()
        skewness = datos.skew()  # Nueva métrica: asimetría
        kurtosis = datos.kurtosis()  # Nueva métrica: curtosis

        print(f"📊 Estadísticas Descriptivas para {titulo}")
        print(f"Conteo: {conteo}")
        print(f"Media: {media:.2f}")
        print(f"Mediana: {mediana:.2f}")
        print(f"Moda: {moda:.2f}")
        print(f"Varianza: {varianza:.2f}")
        print(f"Desviación Estándar: {desv_est:.2f}")
        print(f"Mínimo: {minimo:.2f}")
        print(f"Máximo: {maximo:.2f}")
        print(f"Asimetría (Skew): {skewness:.2f}")
        print(f"Curtosis: {kurtosis:.2f}")

# Nueva función para visualización
def visualizar_distribucion(b=None):
    with output:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            col = 'INSTRUMENTO'
            df = df.copy()
            if anio_pf_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value]
            titulo = f"{margen_dropdown.value}: {variable}"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Eléctricos.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_pe_dropdown.value
            col = 'PIEZOMETRO'
            if progresiva_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
            if anio_pe_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
            titulo = f"{progresiva_dropdown.value} – {variable}"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Casagrande.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_cg_dropdown.value
            col = 'PIEZOMETRO'
            if margen_cg_dropdown.value != "Todos":
                df = df[df['MARGEN'] == margen_cg_dropdown.value]
            if pz_cg_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
            if anio_cg_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
            titulo = f"{margen_cg_dropdown.value} – {variable}"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Inclinómetros.")
                return
            df = df.copy()
            df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
            variable = eje_dropdown.value
            col = 'Inclinometro'
            if inclinometro_dropdown.value != "Todos":
                df = df[df['Inclinometro'] == inclinometro_dropdown.value]
            if anio_inc_dropdown.value != "Todos":
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)]
            titulo = f"{inclinometro_dropdown.value} – {variable}"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Celdas de Asentamiento.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_ca_dropdown.value
            col = 'CELDA_DE_ASENTAMIENTO'
            if progresiva_ca_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]
            if celda_dropdown.value != "Todas":
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value]
            if anio_ca_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)]
            titulo = f"{progresiva_ca_dropdown.value} – {variable}"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Freatímetros.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_fr_dropdown.value
            col = 'FREATIMETRO'
            if freatimetro_dropdown.value != "Todos":
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value]
            if anio_fr_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)]
            titulo = f"{freatimetro_dropdown.value} – {variable}"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Extensómetros.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_ex_dropdown.value
            col = 'EXTENSOMETRO'
            if extensometro_dropdown.value != "Todos":
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value]
            if anio_ex_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)]
            titulo = f"{extensometro_dropdown.value} – {variable}"
        else:
            print("⚠️ Instrumento no reconocido.")
            return

        if variable not in df.columns:
            print(f"⚠️ La variable {variable} no está disponible en los datos.")
            return

        df[variable] = pd.to_numeric(df[variable], errors='coerce')
        df = df.dropna(subset=[variable])
        if df.empty:
            print(f"⚠️ No hay datos válidos para la variable {variable} después de filtrar.")
            return

        # Generar gráficos
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        sns.histplot(df[variable], kde=True)
        plt.title(f"Histograma de {titulo}")
        plt.xlabel(variable)
        plt.ylabel("Frecuencia")
        plt.subplot(1, 2, 2)
        sns.boxplot(y=df[variable])
        plt.title(f"Boxplot de {titulo}")
        plt.ylabel(variable)
        plt.tight_layout()
        plt.show()

# Función para regresión lineal
def realizar_regresion(b=None):
    with output:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        fecha_col = 'FECHA'  # Default para la mayoría
        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            col = 'INSTRUMENTO'
            df = df.copy()
            if anio_pf_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value]
            titulo = f"{margen_dropdown.value}: {variable} vs. Tiempo"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Eléctricos.")
                return
            df = df.copy()
            variable = variable_pe_dropdown.value
            col = 'PIEZOMETRO'
            if progresiva_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
            if anio_pe_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
            titulo = f"{progresiva_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Casagrande.")
                return
            df = df.copy()
            variable = variable_cg_dropdown.value
            col = 'PIEZOMETRO'
            if margen_cg_dropdown.value != "Todos":
                df = df[df['MARGEN'] == margen_cg_dropdown.value]
            if pz_cg_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
            if anio_cg_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
            titulo = f"{margen_cg_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Inclinómetros.")
                return
            df = df.copy()
            fecha_col = 'Fecha'  # Inclinómetros usa 'Fecha'
            variable = eje_dropdown.value
            col = 'Inclinometro'
            if inclinometro_dropdown.value != "Todos":
                df = df[df['Inclinometro'] == inclinometro_dropdown.value]
            if anio_inc_dropdown.value != "Todos":
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)]
            titulo = f"{inclinometro_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Celdas de Asentamiento.")
                return
            df = df.copy()
            variable = variable_ca_dropdown.value
            col = 'CELDA_DE_ASENTAMIENTO'
            if progresiva_ca_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]
            if celda_dropdown.value != "Todas":
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value]
            if anio_ca_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)]
            titulo = f"{progresiva_ca_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Freatímetros.")
                return
            df = df.copy()
            variable = variable_fr_dropdown.value
            col = 'FREATIMETRO'
            if freatimetro_dropdown.value != "Todos":
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value]
            if anio_fr_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)]
            titulo = f"{freatimetro_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Extensómetros.")
                return
            df = df.copy()
            variable = variable_ex_dropdown.value
            col = 'EXTENSOMETRO'
            if extensometro_dropdown.value != "Todos":
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value]
            if anio_ex_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)]
            titulo = f"{extensometro_dropdown.value} – {variable} vs. Tiempo"
        else:
            print("⚠️ Instrumento no reconocido.")
            return

        if variable not in df.columns or fecha_col not in df.columns:
            print(f"⚠️ La variable {variable} o la columna de fecha no están disponibles en los datos.")
            return

        df[variable] = pd.to_numeric(df[variable], errors='coerce')
        df[fecha_col] = pd.to_datetime(df[fecha_col], dayfirst=True, errors='coerce')
        df = df.dropna(subset=[variable, fecha_col])
        if df.empty or len(df) < 2:
            print(f"⚠️ No hay suficientes datos válidos para la regresión de {variable}.")
            return

        # Normalizar Dias para mejorar estabilidad numérica
        df['Dias'] = (df[fecha_col] - df[fecha_col].min()).dt.total_seconds() / (24 * 3600)
        if df['Dias'].std() > 0:
            df['Dias'] = (df['Dias'] - df['Dias'].mean()) / df['Dias'].std()
        else:
            df['Dias'] = df['Dias'] - df['Dias'].mean()  # Centrar si std es 0
        X = sm.add_constant(df['Dias'])
        y = df[variable]

        # Ajustar modelo
        try:
            model = sm.OLS(y, X).fit()
        except Exception as e:
            print(f"❌ Error al ajustar el modelo de regresión: {e}")
            return

        # Predicciones
        y_pred = model.predict(X)

        # Mostrar resultados
        print(f"📈 Regresión Lineal para {titulo}")
        print(model.summary())

        # Visualización
        plt.figure(figsize=(10, 6))
        plt.scatter(df[fecha_col], y, label='Datos', color='blue')
        plt.plot(df[fecha_col], y_pred, label='Regresión', color='red')
        plt.xlabel('Fecha')
        plt.ylabel(variable)
        plt.title(titulo)
        plt.legend()
        plt.grid(True)
        plt.show()

# Conectar eventos
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
margen_dropdown.observe(actualizar_opciones_pf, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
margen_cg_dropdown.observe(actualizar_piezometros_cg, names='value')
progresiva_ca_dropdown.observe(actualizar_celdas_ca, names='value')
boton_estadisticas.on_click(calcular_estadisticas)
boton_regresion.on_click(realizar_regresion)
boton_visualizar.on_click(visualizar_distribucion)  # Conectar nuevo botón

# Mostrar interfaz
display(HTML("<h3 style='color:#1866a3'>Análisis Estadístico y Regresión</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))
display(widgets.HBox([
    margen_dropdown, punto_dropdown,
    progresiva_dropdown, piezometro_dropdown,
    margen_cg_dropdown, pz_cg_dropdown,
    inclinometro_dropdown,
    progresiva_ca_dropdown, celda_dropdown,
    freatimetro_dropdown,
    extensometro_dropdown
]))
display(widgets.HBox([
    variable_pf_dropdown, variable_pe_dropdown, variable_cg_dropdown,
    anio_pf_dropdown, anio_pe_dropdown, anio_cg_dropdown,
    anio_inc_dropdown, eje_dropdown,
    variable_ca_dropdown, anio_ca_dropdown,
    variable_fr_dropdown, anio_fr_dropdown,
    variable_ex_dropdown, anio_ex_dropdown
]))
display(widgets.HBox([boton_estadisticas, boton_regresion, boton_visualizar]))  # Añadir botón visualizar
display(output)

# Inicializar controles
actualizar_controles_visibles()


Output()

In [12]:
import pandas as pd
import numpy as np
from scipy import stats
import ipywidgets as widgets
from IPython.display import display, clear_output, HTML
import statsmodels.api as sm
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

# Asumimos que datos_csv y datos_xlsx están poblados por el script de carga
instrumentos = [
    "puntos_fijos_mi", "puntos_fijos_md", "inclinometros", "asentamiento",
    "piezometros_electricos", "piezometros_casagrande", "freatimetros", "extensometro"
]

try:
    datos_csv
except NameError:
    datos_csv = {inst: pd.DataFrame() for inst in instrumentos}
try:
    datos_xlsx
except NameError:
    datos_xlsx = {inst: pd.DataFrame() for inst in instrumentos}

# Definir widgets
instrumento_dropdown = widgets.Dropdown(
    options=["Puntos Fijos", "Piezómetros Eléctricos", "Piezómetros Casagrande", "Inclinómetros",
             "Celdas de Asentamiento", "Freatímetros", "Extensómetros"],
    value="Puntos Fijos",
    description="Instrumento:"
)
origen_dropdown = widgets.Dropdown(options=["CSV", "XLSX"], value="CSV", description="Origen:")
margen_dropdown = widgets.Dropdown(description="Margen:", options=[])
punto_dropdown = widgets.Dropdown(description="Punto Fijo:", options=[])
variable_pf_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_pf_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
progresiva_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
piezometro_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_pe_dropdown = widgets.Dropdown(description="Variable:", options=[])
variables_explicativas_pe_dropdown = widgets.Dropdown(description="Variables Explicativas:", options=["Ninguna"], multiple=True)
anio_pe_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
margen_cg_dropdown = widgets.Dropdown(description="Margen:", options=[])
pz_cg_dropdown = widgets.Dropdown(description="Piezómetro:", options=[])
variable_cg_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_cg_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
inclinometro_dropdown = widgets.Dropdown(description="Inclinómetro:", options=[])
anio_inc_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
eje_dropdown = widgets.Dropdown(options=["A+", "A-", "B+", "B-"], value="A+", description="Eje:")
progresiva_ca_dropdown = widgets.Dropdown(description="Progresiva:", options=[])
celda_dropdown = widgets.Dropdown(description="Celda:", options=[])
variable_ca_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ca_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
freatimetro_dropdown = widgets.Dropdown(description="Freatímetro:", options=[])
variable_fr_dropdown = widgets.Dropdown(description="Variable:", options=[])
variables_explicativas_fr_dropdown = widgets.Dropdown(description="Variables Explicativas:", options=["Ninguna"], multiple=True)
anio_fr_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])
extensometro_dropdown = widgets.Dropdown(description="Extensómetro:", options=[])
variable_ex_dropdown = widgets.Dropdown(description="Variable:", options=[])
anio_ex_dropdown = widgets.Dropdown(description="Año:", options=["Todos"])

# Botones y salida
boton_estadisticas = widgets.Button(description="Calcular Estadísticas", button_style="primary")
boton_regresion = widgets.Button(description="Realizar Regresión", button_style="success")
boton_visualizar = widgets.Button(description="Visualizar Distribución", button_style="info")
output = widgets.Output()

# Funciones para actualizar opciones de dropdowns
def actualizar_opciones_pf(change=None):
    origen = origen_dropdown.value
    df_mi = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
    df_md = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
    datasets = {"Margen Izquierda (MI)": df_mi, "Margen Derecha (MD)": df_md}
    datasets = {k: v for k, v in datasets.items() if not v.empty}
    if not datasets:
        margen_dropdown.options = []
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = ["Todos"]
        return
    margen_dropdown.options = list(datasets.keys())
    margen = margen_dropdown.value or list(datasets.keys())[0]
    df = datasets[margen].copy()
    if 'FECHA' not in df.columns:
        punto_dropdown.options = []
        variable_pf_dropdown.options = []
        anio_pf_dropdown.options = ["Todos"]
        return
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    variable_pf_dropdown.options = [col for col in df.select_dtypes(include='number').columns if col not in ['FECHA', 'INSTRUMENTO', 'MARGEN']]
    punto_dropdown.options = ["Todos"] + sorted(df['INSTRUMENTO'].dropna().unique().tolist())
    anio_pf_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_opciones_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'FECHA' not in df.columns:
        progresiva_dropdown.options = []
        piezometro_dropdown.options = []
        variable_pe_dropdown.options = []
        variables_explicativas_pe_dropdown.options = ["Ninguna"]
        anio_pe_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique().tolist())
    if progresiva_dropdown.options:
        progresiva_dropdown.value = progresiva_dropdown.options[0]
    actualizar_piezometros_pe()
    variable_pe_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'PIEZOMETRO']]
    variables_explicativas_pe_dropdown.options = ["Ninguna"] + [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'PIEZOMETRO', variable_pe_dropdown.value or '']]
    anio_pe_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_pe(change=None):
    df = datos_csv["piezometros_electricos"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_electricos"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'PIEZOMETRO' not in df.columns:
        piezometro_dropdown.options = []
        return
    piezos = sorted(df[df['PROGRESIVA'] == progresiva_dropdown.value]['PIEZOMETRO'].dropna().unique().tolist())
    piezometro_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    if df.empty or 'MARGEN' not in df.columns or 'FECHA' not in df.columns:
        margen_cg_dropdown.options = []
        pz_cg_dropdown.options = []
        variable_cg_dropdown.options = []
        anio_cg_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    margen_cg_dropdown.options = sorted(df['MARGEN'].dropna().unique().tolist())
    if margen_cg_dropdown.options:
        margen_cg_dropdown.value = margen_cg_dropdown.options[0]
    actualizar_piezometros_cg()
    variable_cg_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'MARGEN', 'PIEZOMETRO']]
    anio_cg_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_piezometros_cg(change=None):
    df = datos_csv["piezometros_casagrande"] if origen_dropdown.value == "CSV" else datos_xlsx["piezometros_casagrande"]
    if df.empty or 'MARGEN' not in df.columns or 'PIEZOMETRO' not in df.columns:
        pz_cg_dropdown.options = []
        return
    piezos = sorted(df[df['MARGEN'] == margen_cg_dropdown.value]['PIEZOMETRO'].dropna().unique().tolist())
    pz_cg_dropdown.options = ["Todos"] + list(piezos)

def actualizar_opciones_inc(change=None):
    df = datos_csv["inclinometros"] if origen_dropdown.value == "CSV" else datos_xlsx["inclinometros"]
    if df.empty or 'Inclinometro' not in df.columns or 'Fecha' not in df.columns:
        inclinometro_dropdown.options = []
        anio_inc_dropdown.options = ["Todos"]
        eje_dropdown.options = ["A+", "A-", "B+", "B-"]
        return
    df = df.copy()
    df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
    inclinometro_dropdown.options = ["Todos"] + sorted(df['Inclinometro'].dropna().unique().tolist())
    anio_inc_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['Fecha'].dt.year.dropna().unique())]
    eje_dropdown.options = ["A+", "A-", "B+", "B-"]

def actualizar_opciones_ca(change=None):
    df = datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'FECHA' not in df.columns:
        progresiva_ca_dropdown.options = []
        celda_dropdown.options = []
        variable_ca_dropdown.options = []
        anio_ca_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    progresiva_ca_dropdown.options = sorted(df['PROGRESIVA'].dropna().unique().tolist())
    if progresiva_ca_dropdown.options:
        progresiva_ca_dropdown.value = progresiva_ca_dropdown.options[0]
    actualizar_celdas_ca()
    variable_ca_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'PROGRESIVA', 'CELDA_DE_ASENTAMIENTO']]
    anio_ca_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_celdas_ca(change=None):
    df = datos_csv["asentamiento"] if origen_dropdown.value == "CSV" else datos_xlsx["asentamiento"]
    if df.empty or 'PROGRESIVA' not in df.columns or 'CELDA_DE_ASENTAMIENTO' not in df.columns:
        celda_dropdown.options = []
        return
    celdas = sorted(df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]['CELDA_DE_ASENTAMIENTO'].dropna().unique().tolist())
    celda_dropdown.options = ["Todas"] + list(celdas)

def actualizar_opciones_fr(change=None):
    df = datos_csv["freatimetros"] if origen_dropdown.value == "CSV" else datos_xlsx["freatimetros"]
    if df.empty or 'FREATIMETRO' not in df.columns or 'FECHA' not in df.columns:
        freatimetro_dropdown.options = []
        variable_fr_dropdown.options = []
        variables_explicativas_fr_dropdown.options = ["Ninguna"]
        anio_fr_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    freatimetro_dropdown.options = ["Todos"] + sorted(df['FREATIMETRO'].dropna().unique().tolist())
    variable_fr_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'FREATIMETRO']]
    variables_explicativas_fr_dropdown.options = ["Ninguna"] + [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'FREATIMETRO', variable_fr_dropdown.value or '']]
    anio_fr_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

def actualizar_opciones_ex(change=None):
    df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
    if df.empty or 'EXTENSOMETRO' not in df.columns or 'FECHA' not in df.columns:
        extensometro_dropdown.options = []
        variable_ex_dropdown.options = []
        anio_ex_dropdown.options = ["Todos"]
        return
    df = df.copy()
    df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
    extensometro_dropdown.options = ["Todos"] + sorted(df['EXTENSOMETRO'].dropna().unique().tolist())
    variable_ex_dropdown.options = [c for c in df.select_dtypes(include='number').columns if c not in ['FECHA', 'EXTENSOMETRO']]
    anio_ex_dropdown.options = ["Todos"] + [str(y) for y in sorted(df['FECHA'].dt.year.dropna().unique())]

# Función para mostrar/ocultar controles
def actualizar_controles_visibles(change=None):
    tipo = instrumento_dropdown.value
    for w in [margen_dropdown, punto_dropdown, variable_pf_dropdown, anio_pf_dropdown,
              progresiva_dropdown, piezometro_dropdown, variable_pe_dropdown, variables_explicativas_pe_dropdown, anio_pe_dropdown,
              margen_cg_dropdown, pz_cg_dropdown, variable_cg_dropdown, anio_cg_dropdown,
              inclinometro_dropdown, anio_inc_dropdown, eje_dropdown,
              progresiva_ca_dropdown, celda_dropdown, variable_ca_dropdown, anio_ca_dropdown,
              freatimetro_dropdown, variable_fr_dropdown, variables_explicativas_fr_dropdown, anio_fr_dropdown,
              extensometro_dropdown, variable_ex_dropdown, anio_ex_dropdown]:
        w.layout.display = 'none'
    if tipo == "Puntos Fijos":
        margen_dropdown.layout.display = 'flex'
        punto_dropdown.layout.display = 'flex'
        variable_pf_dropdown.layout.display = 'flex'
        anio_pf_dropdown.layout.display = 'flex'
        actualizar_opciones_pf()
    elif tipo == "Piezómetros Eléctricos":
        progresiva_dropdown.layout.display = 'flex'
        piezometro_dropdown.layout.display = 'flex'
        variable_pe_dropdown.layout.display = 'flex'
        variables_explicativas_pe_dropdown.layout.display = 'flex'
        anio_pe_dropdown.layout.display = 'flex'
        actualizar_opciones_pe()
    elif tipo == "Piezómetros Casagrande":
        margen_cg_dropdown.layout.display = 'flex'
        pz_cg_dropdown.layout.display = 'flex'
        variable_cg_dropdown.layout.display = 'flex'
        anio_cg_dropdown.layout.display = 'flex'
        actualizar_opciones_cg()
    elif tipo == "Inclinómetros":
        inclinometro_dropdown.layout.display = 'flex'
        anio_inc_dropdown.layout.display = 'flex'
        eje_dropdown.layout.display = 'flex'
        actualizar_opciones_inc()
    elif tipo == "Celdas de Asentamiento":
        progresiva_ca_dropdown.layout.display = 'flex'
        celda_dropdown.layout.display = 'flex'
        variable_ca_dropdown.layout.display = 'flex'
        anio_ca_dropdown.layout.display = 'flex'
        actualizar_opciones_ca()
    elif tipo == "Freatímetros":
        freatimetro_dropdown.layout.display = 'flex'
        variable_fr_dropdown.layout.display = 'flex'
        variables_explicativas_fr_dropdown.layout.display = 'flex'
        anio_fr_dropdown.layout.display = 'flex'
        actualizar_opciones_fr()
    elif tipo == "Extensómetros":
        extensometro_dropdown.layout.display = 'flex'
        variable_ex_dropdown.layout.display = 'flex'
        anio_ex_dropdown.layout.display = 'flex'
        actualizar_opciones_ex()

# Función para calcular estadísticas
def calcular_estadisticas(b=None):
    with output:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            col = 'INSTRUMENTO'
            df = df.copy()
            if anio_pf_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value]
            titulo = f"{margen_dropdown.value}: {variable}"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Eléctricos.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_pe_dropdown.value
            col = 'PIEZOMETRO'
            if progresiva_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
            if anio_pe_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
            titulo = f"{progresiva_dropdown.value} – {variable}"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Casagrande.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_cg_dropdown.value
            col = 'PIEZOMETRO'
            if margen_cg_dropdown.value != "Todos":
                df = df[df['MARGEN'] == margen_cg_dropdown.value]
            if pz_cg_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
            if anio_cg_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
            titulo = f"{margen_cg_dropdown.value} – {variable}"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Inclinómetros.")
                return
            df = df.copy()
            df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
            variable = eje_dropdown.value
            col = 'Inclinometro'
            if inclinometro_dropdown.value != "Todos":
                df = df[df['Inclinometro'] == inclinometro_dropdown.value]
            if anio_inc_dropdown.value != "Todos":
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)]
            titulo = f"{inclinometro_dropdown.value} – {variable}"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Celdas de Asentamiento.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_ca_dropdown.value
            col = 'CELDA_DE_ASENTAMIENTO'
            if progresiva_ca_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]
            if celda_dropdown.value != "Todas":
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value]
            if anio_ca_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)]
            titulo = f"{progresiva_ca_dropdown.value} – {variable}"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Freatímetros.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_fr_dropdown.value
            col = 'FREATIMETRO'
            if freatimetro_dropdown.value != "Todos":
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value]
            if anio_fr_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)]
            titulo = f"{freatimetro_dropdown.value} – {variable}"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Extensómetros.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_ex_dropdown.value
            col = 'EXTENSOMETRO'
            if extensometro_dropdown.value != "Todos":
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value]
            if anio_ex_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)]
            titulo = f"{extensometro_dropdown.value} – {variable}"
        else:
            print("⚠️ Instrumento no reconocido.")
            return

        if variable not in df.columns:
            print(f"⚠️ La variable {variable} no está disponible en los datos.")
            return

        df[variable] = pd.to_numeric(df[variable], errors='coerce')
        df = df.dropna(subset=[variable])
        if df.empty:
            print(f"⚠️ No hay datos válidos para la variable {variable} después de filtrar.")
            return

        datos = df[variable]
        conteo = datos.count()  # Definir conteo aquí
        media = datos.mean()
        mediana = datos.median()
        moda = stats.mode(datos, keepdims=True)[0][0] if len(datos) > 0 else np.nan
        varianza = datos.var()
        desv_est = datos.std()
        minimo = datos.min()
        maximo = datos.max()
        skewness = datos.skew()
        kurtosis = datos.kurtosis()

        print(f"📊 Estadísticas Descriptivas para {titulo}")
        print(f"Conteo: {conteo}")
        print(f"Media: {media:.2f}")
        print(f"Mediana: {mediana:.2f}")
        print(f"Moda: {moda:.2f}")
        print(f"Varianza: {varianza:.2f}")
        print(f"Desviación Estándar: {desv_est:.2f}")
        print(f"Mínimo: {minimo:.2f}")
        print(f"Máximo: {maximo:.2f}")
        print(f"Asimetría (Skew): {skewness:.2f}")
        print(f"Curtosis: {kurtosis:.2f}")

# Función para visualización
def visualizar_distribucion(b=None):
    with output:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            col = 'INSTRUMENTO'
            df = df.copy()
            if anio_pf_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value]
            titulo = f"{margen_dropdown.value}: {variable}"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Eléctricos.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_pe_dropdown.value
            col = 'PIEZOMETRO'
            if progresiva_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
            if anio_pe_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
            titulo = f"{progresiva_dropdown.value} – {variable}"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Casagrande.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_cg_dropdown.value
            col = 'PIEZOMETRO'
            if margen_cg_dropdown.value != "Todos":
                df = df[df['MARGEN'] == margen_cg_dropdown.value]
            if pz_cg_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
            if anio_cg_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
            titulo = f"{margen_cg_dropdown.value} – {variable}"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Inclinómetros.")
                return
            df = df.copy()
            df['Fecha'] = pd.to_datetime(df['Fecha'], dayfirst=True, errors='coerce')
            variable = eje_dropdown.value
            col = 'Inclinometro'
            if inclinometro_dropdown.value != "Todos":
                df = df[df['Inclinometro'] == inclinometro_dropdown.value]
            if anio_inc_dropdown.value != "Todos":
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)]
            titulo = f"{inclinometro_dropdown.value} – {variable}"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Celdas de Asentamiento.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_ca_dropdown.value
            col = 'CELDA_DE_ASENTAMIENTO'
            if progresiva_ca_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]
            if celda_dropdown.value != "Todas":
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value]
            if anio_ca_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)]
            titulo = f"{progresiva_ca_dropdown.value} – {variable}"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Freatímetros.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_fr_dropdown.value
            col = 'FREATIMETRO'
            if freatimetro_dropdown.value != "Todos":
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value]
            if anio_fr_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)]
            titulo = f"{freatimetro_dropdown.value} – {variable}"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Extensómetros.")
                return
            df = df.copy()
            df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
            variable = variable_ex_dropdown.value
            col = 'EXTENSOMETRO'
            if extensometro_dropdown.value != "Todos":
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value]
            if anio_ex_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)]
            titulo = f"{extensometro_dropdown.value} – {variable}"
        else:
            print("⚠️ Instrumento no reconocido.")
            return

        if variable not in df.columns:
            print(f"⚠️ La variable {variable} no está disponible en los datos.")
            return

        df[variable] = pd.to_numeric(df[variable], errors='coerce')
        df = df.dropna(subset=[variable])
        if df.empty:
            print(f"⚠️ No hay datos válidos para la variable {variable} después de filtrar.")
            return

        # Generar gráficos
        plt.figure(figsize=(12, 5))
        plt.subplot(1, 2, 1)
        sns.histplot(df[variable], kde=True)
        plt.title(f"Histograma de {titulo}")
        plt.xlabel(variable)
        plt.ylabel("Frecuencia")
        plt.subplot(1, 2, 2)
        sns.boxplot(y=df[variable])
        plt.title(f"Boxplot de {titulo}")
        plt.ylabel(variable)
        plt.tight_layout()
        plt.show()

# Función para regresión lineal multivariable
def realizar_regresion(b=None):
    with output:
        clear_output(wait=True)
        tipo = instrumento_dropdown.value
        origen = origen_dropdown.value
        fecha_col = 'FECHA'
        if tipo == "Puntos Fijos":
            df = datos_csv["puntos_fijos_mi"] if origen == "CSV" else datos_xlsx["puntos_fijos_mi"]
            if margen_dropdown.value == "Margen Derecha (MD)":
                df = datos_csv["puntos_fijos_md"] if origen == "CSV" else datos_xlsx["puntos_fijos_md"]
            variable = variable_pf_dropdown.value
            col = 'INSTRUMENTO'
            explicativas = []
            df = df.copy()
            if anio_pf_dropdown.value != "Todos":
                df['FECHA'] = pd.to_datetime(df['FECHA'], dayfirst=True, errors='coerce')
                df = df[df['FECHA'].dt.year == int(anio_pf_dropdown.value)]
            if punto_dropdown.value != "Todos":
                df = df[df['INSTRUMENTO'] == punto_dropdown.value]
            titulo = f"{margen_dropdown.value}: {variable} vs. Tiempo"
        elif tipo == "Piezómetros Eléctricos":
            df = datos_csv["piezometros_electricos"] if origen == "CSV" else datos_xlsx["piezometros_electricos"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Eléctricos.")
                return
            df = df.copy()
            variable = variable_pe_dropdown.value
            explicativas = variables_explicativas_pe_dropdown.value if variables_explicativas_pe_dropdown.value != ("Ninguna",) else []
            col = 'PIEZOMETRO'
            if progresiva_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_dropdown.value]
            if piezometro_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == piezometro_dropdown.value]
            if anio_pe_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_pe_dropdown.value)]
            titulo = f"{progresiva_dropdown.value} – {variable} vs. Tiempo + {', '.join(explicativas) if explicativas else ''}"
        elif tipo == "Piezómetros Casagrande":
            df = datos_csv["piezometros_casagrande"] if origen == "CSV" else datos_xlsx["piezometros_casagrande"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Piezómetros Casagrande.")
                return
            df = df.copy()
            variable = variable_cg_dropdown.value
            explicativas = []
            col = 'PIEZOMETRO'
            if margen_cg_dropdown.value != "Todos":
                df = df[df['MARGEN'] == margen_cg_dropdown.value]
            if pz_cg_dropdown.value != "Todos":
                df = df[df['PIEZOMETRO'] == pz_cg_dropdown.value]
            if anio_cg_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_cg_dropdown.value)]
            titulo = f"{margen_cg_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Inclinómetros":
            df = datos_csv["inclinometros"] if origen == "CSV" else datos_xlsx["inclinometros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Inclinómetros.")
                return
            df = df.copy()
            fecha_col = 'Fecha'
            variable = eje_dropdown.value
            explicativas = []
            col = 'Inclinometro'
            if inclinometro_dropdown.value != "Todos":
                df = df[df['Inclinometro'] == inclinometro_dropdown.value]
            if anio_inc_dropdown.value != "Todos":
                df = df[df['Fecha'].dt.year == int(anio_inc_dropdown.value)]
            titulo = f"{inclinometro_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Celdas de Asentamiento":
            df = datos_csv["asentamiento"] if origen == "CSV" else datos_xlsx["asentamiento"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Celdas de Asentamiento.")
                return
            df = df.copy()
            variable = variable_ca_dropdown.value
            explicativas = []
            col = 'CELDA_DE_ASENTAMIENTO'
            if progresiva_ca_dropdown.value != "Todos":
                df = df[df['PROGRESIVA'] == progresiva_ca_dropdown.value]
            if celda_dropdown.value != "Todas":
                df = df[df['CELDA_DE_ASENTAMIENTO'] == celda_dropdown.value]
            if anio_ca_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ca_dropdown.value)]
            titulo = f"{progresiva_ca_dropdown.value} – {variable} vs. Tiempo"
        elif tipo == "Freatímetros":
            df = datos_csv["freatimetros"] if origen == "CSV" else datos_xlsx["freatimetros"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Freatímetros.")
                return
            df = df.copy()
            variable = variable_fr_dropdown.value
            explicativas = variables_explicativas_fr_dropdown.value if variables_explicativas_fr_dropdown.value != ("Ninguna",) else []
            col = 'FREATIMETRO'
            if freatimetro_dropdown.value != "Todos":
                df = df[df['FREATIMETRO'] == freatimetro_dropdown.value]
            if anio_fr_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_fr_dropdown.value)]
            titulo = f"{freatimetro_dropdown.value} – {variable} vs. Tiempo + {', '.join(explicativas) if explicativas else ''}"
        elif tipo == "Extensómetros":
            df = datos_csv["extensometro"] if origen == "CSV" else datos_xlsx["extensometro"]
            if df.empty:
                print("⚠️ No hay datos disponibles para Extensómetros.")
                return
            df = df.copy()
            variable = variable_ex_dropdown.value
            explicativas = []
            col = 'EXTENSOMETRO'
            if extensometro_dropdown.value != "Todos":
                df = df[df['EXTENSOMETRO'] == extensometro_dropdown.value]
            if anio_ex_dropdown.value != "Todos":
                df = df[df['FECHA'].dt.year == int(anio_ex_dropdown.value)]
            titulo = f"{extensometro_dropdown.value} – {variable} vs. Tiempo"
        else:
            print("⚠️ Instrumento no reconocido.")
            return

        if variable not in df.columns or fecha_col not in df.columns:
            print(f"⚠️ La variable {variable} o la columna de fecha no están disponibles en los datos.")
            return

        df[variable] = pd.to_numeric(df[variable], errors='coerce')
        df[fecha_col] = pd.to_datetime(df[fecha_col], dayfirst=True, errors='coerce')
        for var in explicativas:
            df[var] = pd.to_numeric(df[var], errors='coerce')
        df = df.dropna(subset=[variable, fecha_col] + explicativas)
        if df.empty or len(df) < 2:
            print(f"⚠️ No hay suficientes datos válidos para la regresión de {variable}.")
            return

        # Preparar variables explicativas
        df['Dias'] = (df[fecha_col] - df[fecha_col].min()).dt.total_seconds() / (24 * 3600)
        X = pd.DataFrame({'const': 1, 'Dias': df['Dias']})
        for var in explicativas:
            X[var] = df[var]
        # Normalizar variables explicativas
        for col in X.columns:
            if col != 'const' and X[col].std() > 0:
                X[col] = (X[col] - X[col].mean()) / X[col].std()
            elif col != 'const':
                X[col] = X[col] - X[col].mean()
        y = df[variable]

        # Ajustar modelo
        try:
            model = sm.OLS(y, X).fit()
        except Exception as e:
            print(f"❌ Error al ajustar el modelo de regresión: {e}")
            return

        # Predicciones
        y_pred = model.predict(X)

        # Mostrar resultados
        print(f"📈 Regresión Lineal para {titulo}")
        print(model.summary())

        # Visualización
        if len(explicativas) == 0:
            plt.figure(figsize=(10, 6))
            plt.scatter(df[fecha_col], y, label='Datos', color='blue')
            plt.plot(df[fecha_col], y_pred, label='Regresión', color='red')
            plt.xlabel('Fecha')
            plt.ylabel(variable)
            plt.title(titulo)
            plt.legend()
            plt.grid(True)
            plt.show()
        elif len(explicativas) == 1:
            fig = px.scatter_3d(df, x='Dias', y=explicativas[0], z=variable, color=y_pred)
            fig.update_layout(title=titulo, scene=dict(xaxis_title="Días", yaxis_title=explicativas[0], zaxis_title=variable))
            fig.show()

# Conectar eventos
instrumento_dropdown.observe(actualizar_controles_visibles, names='value')
origen_dropdown.observe(actualizar_controles_visibles, names='value')
margen_dropdown.observe(actualizar_opciones_pf, names='value')
progresiva_dropdown.observe(actualizar_piezometros_pe, names='value')
margen_cg_dropdown.observe(actualizar_piezometros_cg, names='value')
progresiva_ca_dropdown.observe(actualizar_celdas_ca, names='value')
boton_estadisticas.on_click(calcular_estadisticas)
boton_regresion.on_click(realizar_regresion)
boton_visualizar.on_click(visualizar_distribucion)

# Mostrar interfaz
display(HTML("<h3 style='color:#1866a3'>Análisis Estadístico y Regresión</h3>"))
display(widgets.HBox([instrumento_dropdown, origen_dropdown]))
display(widgets.HBox([
    margen_dropdown, punto_dropdown,
    progresiva_dropdown, piezometro_dropdown,
    margen_cg_dropdown, pz_cg_dropdown,
    inclinometro_dropdown,
    progresiva_ca_dropdown, celda_dropdown,
    freatimetro_dropdown,
    extensometro_dropdown
]))
display(widgets.HBox([
    variable_pf_dropdown, variable_pe_dropdown, variable_cg_dropdown,
    anio_pf_dropdown, anio_pe_dropdown, anio_cg_dropdown,
    anio_inc_dropdown, eje_dropdown,
    variable_ca_dropdown, anio_ca_dropdown,
    variable_fr_dropdown, variables_explicativas_fr_dropdown, anio_fr_dropdown,
    variable_ex_dropdown, anio_ex_dropdown,
    variables_explicativas_pe_dropdown
]))
display(widgets.HBox([boton_estadisticas, boton_regresion, boton_visualizar]))
display(output)

# Inicializar controles
actualizar_controles_visibles()


Output()